<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Imports-and-Update-The-Latest-Data" data-toc-modified-id="Imports-and-Update-The-Latest-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports and Update The Latest Data</a></span></li><li><span><a href="#Historical-air-4-Thai-Data" data-toc-modified-id="Historical-air-4-Thai-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Historical air 4 Thai Data</a></span></li><li><span><a href="#Power-Plants" data-toc-modified-id="Power-Plants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Power Plants</a></span></li></ul></div>

# Data Source 
1. Berekely Earth 'http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/'
2. Screaped Air4Thai Data 'http://air4thai.pcd.go.th/webV2/history/'
3. CDC Data 'https://www.cmuccdc.org/download_json/'
4. Old Air4Thai data from Thailand EPA

# Imports and Update The Latest Data

In [81]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.read_data import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
b_folder='../data/pm25/'
a4th_folder='../data/air4thai_hourly/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'

In [3]:
# Data from Berkeley Earth Projects: 
download_b_data(data_folder='../data/pm25/', url='http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/')
get_city_info(data_folder='../data/pm25/')

100% [............................................................................] 793700 / 793700

In [85]:
download_cdc_data(station_url='https://www.cmuccdc.org/api/ccdc/stations', 
                  dl_url= 'https://www.cmuccdc.org/download_json/', 
                  data_folder='../data/cdc_data/')

5246
5275
4036
6014
5208
4048
6015
4067
122
5373
5374
4033
6013
4062
123
5093
4035
4038
121
5071
5262
5268
5029
5266
4044
5378
116
6006
5151
5379
6008
5278
5066
5277
5167
5161
5193
75
5206
5285
5194
5283
5284
6011
5269
6005
6004
6009
6010
6007
5286
5195
5300
5221
20
6002
5211
5276
5227
5181
5200
47
5013
5008
5018
5281
5287
5012
5015
5036
5273
5022
4018
4013
4014
5264
4019
5072
5070
4022
5073
22
5083
5074
4020
5084
5085
80
4021
4002
64
74
4023
9
5096
5154
5279
4047
5020
5069
5035
5061
6016
6017
6018
5041
6003
5086
119
5315
79
6000
5341
5339
5321
95
5328
5333
117
5330
5331
45
89
43
72
78
81
5270
5004
44
5002
94
100
5336
71
93
5267
4026
69
5263
5265
36
104
5377
6020
6019
6022
4027
4031
4049
6021
4037
6012
5272
5274
5376


In [72]:
update_last_air4Thai(url='http://air4thai.pcd.go.th/webV2/history/',data_folder='../data/air4thai_hourly/')



0it [00:00, ?it/s]

['02t', '03t', '05t', '08t', '10t', '11t', '12t', '13t', '14t', '16t', '17t', '18t', '19t', '20t', '21t', '22t', '24t', '25t', '26t', '27t', '28t', '29t', '30t', '31t', '32t', '33t', '34t', '35t', '36t', '37t', '38t', '39t', '40t', '41t', '42t', '43t', '44t', '46t', '47t', '50t', '52t', '53t', '54t', '57t', '58t', '59t', '60t', '61t', '62t', '63t', '67t', '68t', '69t', '70t', '71t', '72t', '73t', '74t', '75t', '76t', '77t', '79t', '80t', '81t', '82t', '83t', '84t', 'm1', 'm4', 'm8', 'm9', 'o10', 'o20', 'o22', 'o23', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29']




1it [00:18, 18.75s/it]

2it [00:37, 18.76s/it]

3it [00:56, 18.77s/it]

4it [01:15, 18.80s/it]

5it [01:33, 18.78s/it]

6it [01:52, 18.81s/it]

7it [02:11, 18.82s/it]

8it [02:30, 18.80s/it]

9it [02:49, 18.85s/it]

10it [03:08, 18.90s/it]

11it [03:27, 18.89s/it]

12it [03:45, 18.84s/it]

13it [04:04, 18.77s/it]

14it [04:23, 18.76s/it]

15it [04:42, 18.80s/it]

16it [05:01, 18.81s/it]

17it [05:20, 18.87s/it]

18it [05:38, 18.89s/it]

19it [05:57, 18.83s/it]

20it [06:16, 18.82s/it]

21it [06:35, 18.79s/it]

22it [06:54, 18.79s/it]

23it [07:12, 18.77s/it]

24it [07:31, 18.79s/it]

25it [07:50, 18.80s/it]

26it [08:09, 18.78s/it]

27it [08:27, 18.80s/it]

28it [08:46, 18.80s/it]

29it [09:05, 18.80s/it]

30it [09:24, 18.82s/it]

31it [09:43, 18.80s/it]

32it [10:02, 18.81s/it]

33it [10:21, 18.95s/it]

34it [10:39, 18.84s/it]

35it [10:58, 18.83s/it]

36it [11:17, 18.82s/it]

37it [11:36, 18.89s/it]

38it [11:55, 18.89s/it]

39it [12:14, 18.89s/it]

40it [12:33, 18.89s/it]

41it [1

create new ../data/air4thai_hourly/57t.csv




45it [14:08, 18.96s/it]

...

create new ../data/air4thai_hourly/o27.csv




80it [25:08, 18.88s/it]

create new ../data/air4thai_hourly/o28.csv




81it [25:27, 18.86s/it]

create new ../data/air4thai_hourly/o29.csv


# Historical air 4 Thai Data

In [63]:
# load stations information
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

# find stations in Chiangmai and parase that files
cm_station_ids = []
for i, stations in enumerate(station_info):
    if 'Chiang Mai' in stations['areaEN']:
        # ignore station that start with o
        if 'o' not in stations['stationID']:
            cm_station_ids.append(stations['stationID'])
print(cm_station_ids)

['35t', '36t', 'm9']


In [29]:
# parase historical data 
for station_id in cm_station_ids:
    # find all files that start with this stations
    p = Path(aqm_folder)
    filenames = []
    for i in p.glob('**/*.xlsx'):
        if station_id in i.name:
            filenames.append(str(i))
        
    # if filename exist load that file
    if len(filenames) >0:

        save_filename = aqm_folder + 'process/' + station_id + '.csv'
        print('save filename', save_filename)
        station_data = read_his_xl(filenames[0])
        #print(station_data.head())

        # save the data if the dataframe is not empty
        if len(station_data)> 0:
            station_data.to_csv(save_filename,index=False)

['35t', '36t', 'm9', 'o10', 'o20', 'o22']
35t
save filename ../data/aqm_hourly2/process/35t.csv
36t
save filename ../data/aqm_hourly2/process/36t.csv
m9
o10
o20
o22


In [45]:
cm_data1 = pd.read_csv(aqm_folder + 'process/35t.csv')
cm_data1['datetime'] = pd.to_datetime(cm_data1['datetime'])
cm_data1 = cm_data1.set_index('datetime')
# keep only gas columns
cm_data1 = cm_data1[['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5']]

cm_data2 = pd.read_csv(aqm_folder + 'process/36t.csv')
cm_data2['datetime'] = pd.to_datetime(cm_data2['datetime'])
cm_data2 = cm_data2.set_index('datetime')


# Power Plants

In [92]:
url = 'https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B9%82%E0%B8%A3%E0%B8%87%E0%B9%84%E0%B8%9F%E0%B8%9F%E0%B9%89%E0%B8%B2%E0%B9%83%E0%B8%99%E0%B9%84%E0%B8%97%E0%B8%A2'
url = 'https://en.wikipedia.org/wiki/List_of_power_stations_in_Thailand'
table_list = pd.read_html(url)
len(table_list)
p_folder = '../data/power_plants/'
for i, table in enumerate(table_list):
    table.to_csv(p_folder + f'table_eng{i}.csv',index=False)